In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

In [2]:
n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()
x = 12
d = 0
y = 34
distances = sc.parallelize([(x, d)]).partitionBy(n)

In [3]:
while True:
  candidates = distances.join(forward_edges, n).map(step)
  new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
  count = new_distances.filter(lambda i: i[1] == d + 1).count()
  filtered = new_distances.filter(lambda i: i[1] == d + 1).distinct().collect()
  if count > 0:
    d += 1
    distances = new_distances.distinct()
    #print ("d = %d count = %d" %(d,count))
    #print filtered
    end_filter = new_distances.filter(lambda i: i[0] == y).count()
    if end_filter > 0:
        break
  else:
    break

In [4]:
#y = 34
#d = 8
result = [y]
while True:
    prev_followers = new_distances.filter(lambda i: i[1] == d - 1).distinct().collect()
    for element in prev_followers:
        prev_follower = sc.parallelize([(element[0], element[1])]).partitionBy(n)
        check_prev = prev_follower.join(forward_edges, n).filter(lambda i: i[1][1] == y).count()
        if check_prev > 0:
            result.insert(0,element[0])
            y = element[0]
            d -= 1
            break
    if d == 0:
        break
myString = ",".join(map(str, result))
print myString

12,422,53,52,107,20,23,274,34
